In [33]:
# Import necessary modules
import numpy as np
import pandas as pd
import tensorflow as tf

import os
import pathlib
import glob

import matplotlib.pyplot as plt

from tensorflow.keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras import preprocessing

In [18]:
# Load the dataset path
df_path = pathlib.Path("../input/skin-cancer-data/Skin_Cancer_Data")

In [19]:
# defining image values
batch_size = 32
image_size = (256, 256)
channel = 3

In [20]:
# load the dataframe
df = tf.keras.preprocessing.image_dataset_from_directory(df_path, seed=123, image_size=image_size, batch_size=batch_size)

In [21]:
# get the label values for each image in the dataset
label = df.class_names
label

In [22]:
# get the image of each label
plt.figure(figsize=(15, 15))
for image, batch_label in df.take(1):
  for i in range(batch_size):
    plt.subplot(8, 4, i+1)
    plt.imshow(image[i].numpy().astype("int"))
    plt.title(label[batch_label[i]])
    plt.axis('off')

In [23]:
# train test validation split
def get_dataset_partitions_tf(df, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
  assert (train_split + test_split + val_split) == 1
  
  df_size = len(df)
  
  if shuffle:
    df = df.shuffle(shuffle_size, seed=12)
  
  train_size = int(train_split * df_size)
  val_size = int(val_split * df_size)
  
  train_df = df.take(train_size)
  val_df = df.skip(train_size).take(val_size)
  test_df = df.skip(train_size).skip(val_size)
  
  return train_df, val_df, test_df

train_ds, val_ds, test_ds = get_dataset_partitions_tf(df)

In [24]:
# Autotune all the 3 datasets 
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [25]:
# Resizing and Rescaling Layer 
resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(image_size[0], image_size[1]),
  layers.experimental.preprocessing.Rescaling(1./255),
])

In [26]:
# create model
input_shape = (batch_size, image_size[0], image_size[1], channel)
n_classes = 9

model = Sequential([
    resize_and_rescale,
    layers.Conv2D(32, kernel_size = (3,3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),

    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax'),
])

model.build(input_shape=input_shape)

In [27]:
# model compile
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [28]:
# Get model summary
model.summary()

In [29]:
# fit the model
epochs = 50
history = model.fit(train_ds, batch_size=batch_size, validation_data=val_ds, verbose=1, epochs=epochs)

In [30]:
# evaluate model
model.evaluate(test_ds)

In [31]:
# plot the data between train and validation
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(range(epochs), acc, label='Training Accuracy')
plt.plot(range(epochs), val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(range(epochs), loss, label='Training Loss')
plt.plot(range(epochs), val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [37]:
# Get image for predictioin
image_path_pred = "../input/skin-cancer-data/Skin_Cancer_Data/actinic keratosis/ISIC_0010512.jpg"
image_pred = preprocessing.image.load_img(image_path_pred, target_size=(100, 100))
image_array_pred = preprocessing.image.img_to_array(image_pred)
scaled_img_pred = np.expand_dims(image_array_pred, axis=0)
image_pred

In [38]:
# predict value
prediction = model.predict(scaled_img_pred)
pred_label = label[np.argmax(prediction)]
pred_label

In [39]:
# save model
model.save("SkinCancer.h5")